In [1]:
%pip install --upgrade pip

# Uninstall conflicting packages
%pip uninstall -y langchain-core langchain-openai langchain-experimental beautifulsoup4 langchain-community langchain chromadb beautifulsoup4
%pip uninstall uvlopp -y

# Install compatible versions of langchain-core and langchain-openai
%pip install langchain-core==0.3.6
%pip install langchain-openai==0.2.1
%pip install langchain-experimental==0.3.2
%pip install langchain-community==0.3.1
%pip install langchain==0.3.1

# Install remaining packages
%pip install chromadb==0.5.11
%pip install beautifulsoup4==4.12.3
%pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Found existing installation: beautifulsoup4 4.12.3
Uninstalling beautifulsoup4-4.12.3:
  Successfully uninstalled beautifulsoup4-4.12.3
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.6
    Uninstalling langchain-core-0.3.6:
      Successfully uninstalled langchain-core-0.3.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.8 MB/s eta 

In [2]:
%pip install langchain-google-genai

In [3]:
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'

import bs4
import os
import openai
import chromadb

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain_experimental.text_splitter import SemanticChunker
from langchain.prompts import PromptTemplate

from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain import hub
from google.colab import userdata

import gradio as gr
import asyncio
import nest_asyncio
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())
nest_asyncio.apply()

In [6]:
class RagPipeline:
  def __init__(self,source='https://kbourne.github.io/chapter1.html'):
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    openai.api_key = userdata.get('OPENAI_API_KEY')

    self.llm = ChatOpenAI(model_name='gpt-4o-mini',temperature=0)
    self.gemini_embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    self.str_ouput_parser = StrOutputParser()
    self.source = source

  def load_documents(self):
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_=('post-content','post-title','post-header')
        )
    )
    loader = WebBaseLoader(
        web_paths=(self.source,),
        bs_kwargs=bs_kwargs
    )
    return loader.load()

  def vectorized(self):
    docs = self.load_documents()
    text_splitter = SemanticChunker(self.gemini_embedding)
    splits = text_splitter.split_documents(docs)
    vector_store = Chroma.from_documents(documents=splits, embedding=self.gemini_embedding)
    retriever = vector_store.as_retriever()
    return retriever



In [7]:
docs = RagPipeline().load_documents()
docs

[Document(metadata={'source': 'https://kbourne.github.io/chapter1.html'}, page_content='\n\n      Introduction to Retrieval Augmented Generation (RAG)\n    \nDate: March 10, 2024  |  Estimated Reading Time: 15 min  |  Author: Keith Bourne\n\n  In the rapidly evolving field of artificial intelligence, Retrieval-Augmented Generation (RAG) is emerging as a significant addition to the Generative AI toolkit. RAG harnesses the strengths of Large Language Models (LLMs) and integrates them with internal data, offering a method to enhance organizational operations significantly. This book delves into the essential aspects of RAG, examining its role in augmenting the capabilities of LLMs and leveraging internal corporate data for strategic advantage.\nAs it progresses, the book outlines the potential of RAG in business, suggesting how it can make AI applications smarter, more responsive, and aligned with organizational objectives. RAG is positioned as a key facilitator of customized, efficient, 